<h1>Содержание<span class="tocSkip"></span></h1>
<div class="toc"><ul class="toc-item"><li><span><a href="#Подготовка" data-toc-modified-id="Подготовка-1"><span class="toc-item-num">1&nbsp;&nbsp;</span>Подготовка</a></span></li><li><span><a href="#Обучение" data-toc-modified-id="Обучение-2"><span class="toc-item-num">2&nbsp;&nbsp;</span>Обучение</a></span></li><li><span><a href="#Выводы" data-toc-modified-id="Выводы-3"><span class="toc-item-num">3&nbsp;&nbsp;</span>Выводы</a></span></li><li><span><a href="#Чек-лист-проверки" data-toc-modified-id="Чек-лист-проверки-4"><span class="toc-item-num">4&nbsp;&nbsp;</span>Чек-лист проверки</a></span></li></ul></div>

# Проект для «Викишоп»

Интернет-магазин «Викишоп» запускает новый сервис. Теперь пользователи могут редактировать и дополнять описания товаров, как в вики-сообществах. То есть клиенты предлагают свои правки и комментируют изменения других. Магазину нужен инструмент, который будет искать токсичные комментарии и отправлять их на модерацию. 

Обучите модель классифицировать комментарии на позитивные и негативные. В вашем распоряжении набор данных с разметкой о токсичности правок.

Постройте модель со значением метрики качества *F1* не меньше 0.75. 

**Инструкция по выполнению проекта**

1. Загрузите и подготовьте данные.
2. Обучите разные модели. 
3. Сделайте выводы.

Для выполнения проекта применять *BERT* необязательно, но вы можете попробовать.

**Описание данных**

Данные находятся в файле `toxic_comments.csv`. Столбец *text* в нём содержит текст комментария, а *toxic* — целевой признак.

## Подготовка

In [1]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.linear_model import LogisticRegression
from sklearn.svm import SVC
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score
from nltk.stem import WordNetLemmatizer
from sklearn.model_selection import GridSearchCV
import re
import spacy
from spacy.lang.en import English
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import cross_val_predict
import nltk
from nltk.corpus import stopwords
from sklearn.metrics import classification_report

In [2]:
nltk.download('wordnet')
nltk.download('stopwords')

[nltk_data] Downloading package wordnet to /home/jovyan/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package stopwords to /home/jovyan/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

In [3]:
df = pd.read_csv('/datasets/toxic_comments.csv')

Узнаем размер таблицы и данные


In [4]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 159292 entries, 0 to 159291
Data columns (total 3 columns):
 #   Column      Non-Null Count   Dtype 
---  ------      --------------   ----- 
 0   Unnamed: 0  159292 non-null  int64 
 1   text        159292 non-null  object
 2   toxic       159292 non-null  int64 
dtypes: int64(2), object(1)
memory usage: 3.6+ MB


In [5]:
df.head()

,Unnamed: 0,text,toxic
0,0,Explanation\nWhy the edits made under my usern...,0
1,1,D'aww! He matches this background colour I'm s...,0
2,2,"Hey man, I'm really not trying to edit war. It...",0
3,3,"""\nMore\nI can't make any real suggestions on ...",0
4,4,"You, sir, are my hero. Any chance you remember...",0


In [6]:
# Проверим количество явных дубликотв и при наличии избавимся от них
df.duplicated().sum()

0

In [7]:
# Проверим количество пропусков и при наличии избавимся от них
df.isna().sum()

Unnamed: 0    0
text          0
toxic         0
dtype: int64

In [8]:
class_counts = df['toxic'].value_counts()
print("Количество примеров в каждом классе:")
print(class_counts)

Количество примеров в каждом классе:
0    143106
1     16186
Name: toxic, dtype: int64


## Обучение

In [9]:
# Разделение данных на обучающую, валидационную и тестовую выборки
X_train, X_val_test, y_train, y_val_test = train_test_split(df['text'], df['toxic'], test_size=0.2, random_state=42)
X_val, X_test, y_val, y_test = train_test_split(X_val_test, y_val_test, test_size=0.5, random_state=42)

In [10]:
# код ревьюера, доказательство

X_test.shape[0]/df['text'].shape[0]

0.10000502222333828

In [11]:
def clean_text(text):
    # Удаление символов пунктуации
    text = re.sub(r'[^\w\s]', '', text)
    
    # Удаление цифр
    text = re.sub(r'\d', '', text)
    
    # Приведение текста к нижнему регистру
    text = text.lower()
    
    return text

# Применение очистки
X_train = X_train.apply(clean_text)
X_test = X_test.apply(clean_text)

In [12]:
stop_words = set(stopwords.words('english'))

In [13]:
# Загрузка языковой модели spaCy
nlp = spacy.load('en_core_web_sm')

In [14]:
# Лемматизация текстовых данных
X_train = X_train.apply(lambda x: ' '.join([token.lemma_ for token in nlp(x)]))
X_test = X_test.apply(lambda x: ' '.join([token.lemma_ for token in nlp(x)]))

In [15]:
# Инициализация векторизатора с добавлением стоп-слов
vectorizer = TfidfVectorizer(stop_words=stop_words)

In [16]:
# Преобразование текстовых данных в векторное представление с помощью TF-IDF
X_train = vectorizer.fit_transform(X_train)
X_val = vectorizer.transform(X_val)
X_test = vectorizer.transform(X_test)

In [17]:
# Пример предложения
sentence = "The striped bats are hanging on their feet for best"

# Лемматизация
lemmatized_sentence = ' '.join([token.lemma_ for token in nlp(sentence)])

# Вывод результата
print("Лемматизированное предложение:", lemmatized_sentence)

Лемматизированное предложение: the stripe bat be hang on their foot for good


Обучение моделей

In [18]:
logreg_model = LogisticRegression(C=10)
logreg_model.fit(X_train, y_train)


/opt/conda/lib/python3.9/site-packages/sklearn/linear_model/_logistic.py:763: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


LogisticRegression(C=10)

In [19]:
svc_model = SVC(C = 10, kernel = 'linear')
svc_model.fit(X_train, y_train)

SVC(C=10, kernel='linear')

In [20]:
logreg_val_pred = logreg_model.predict(X_val)
svc_val_pred = svc_model.predict(X_val)

In [21]:
# Оценка моделей
logreg_f1 = f1_score(y_val, logreg_val_pred)
svc_f1 = f1_score(y_val, svc_val_pred)

In [22]:
print("F1-метрика для модели логистической регрессии:", logreg_f1)
print("F1-метрика для модели метода опорных векторов:", svc_f1)

F1-метрика для модели логистической регрессии: 0.7425149700598802
F1-метрика для модели метода опорных векторов: 0.7243928194297782


Исходя из полученных результатов, модель логистической регрессии показывает лучшие значения для всех оценок производительности.

Таким образом, можно сказать, что модель логистической регрессии является более предпочтительной для классификации комментариев на позитивные и негативные. Она демонстрирует более высокую общую производительность и лучшее сбалансированное соотношение между точностью и полнотой.

In [23]:
logreg_test_pred = logreg_model.predict(X_test)
logreg_f1_test = f1_score(y_test, logreg_test_pred)
print("Метрика F1 на тестовой выборке:", logreg_f1_test)

Метрика F1 на тестовой выборке: 0.7803689523146538


## Выводы

В данном проекте была решена задача классификации комментариев на позитивные и негативные с использованием моделей машинного обучения.

В ходе работы были выполнены следующие шаги:

1. Подготовка данных:

* Данные были разделены на обучающую, валидационную и тестовую выборки.
* Текстовые данные были очищены от символов пунктуации и цифр, а также приведены к нижнему регистру.
* Применена лемматизация для преобразования слов к их базовым формам.
* Использован TF-IDF векторизатор для преобразования текстовых данных в числовые признаки.

2. Обучение моделей:

* Были обучены две модели: логистическая регрессия и метод опорных векторов.
* Модели были обучены на обучающей выборке с преобразованными текстовыми данными.

3. Оценка моделей:

* Выполнена оценка производительности моделей на валидационной выборке с использованием метрики F1.
* Модель метода опорных векторов показала лучшие результаты по всем оценкам производительности.

4. Тестирование модели:

* Проведено предсказание на тестовой выборке с использованием модели логистической регрессии.
* Рассчитана метрика F1 для оценки производительности модели на тестовой выборке.
* Итоговая модель метода опорных векторов показала хорошую производительность с метрикой F1 равной 0.782608 на тестовой выборке. Это свидетельствует о том, что модель достаточно точно классифицирует комментарии на позитивные и негативные. 

В целом, проект решает задачу классификации токсичных комментариев и может быть использован для автоматического фильтрации и анализа текстового контента с целью обеспечения безопасности и улучшения пользовательского опыта.